<a href="https://colab.research.google.com/github/danishrauf46/Sentiment_Analysis_using_LSTM/blob/main/Sentiment_Analysis_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [4]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [5]:
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 17.6MB/s]
100% 25.7M/25.7M [00:02<00:00, 11.4MB/s]


In [7]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [8]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [9]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [10]:
data.shape

(50000, 2)

In [11]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [12]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [13]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [14]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [15]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=2)

In [16]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [17]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [18]:
print(x_train)

[[3474   13  847 ...   78  547  166]
 [   0    0    0 ...  105 3444  176]
 [ 133    6  429 ...  143  155 1198]
 ...
 [ 195  117   32 ...   27    4   91]
 [   0    0    0 ...   19   30  125]
 [  38   88 2252 ...   23   30    9]]


In [19]:
print(x_test)

[[   0    0    0 ...   30   29 2207]
 [  19   13  154 ...   11 3764 2469]
 [   0    0    0 ...  443  223   50]
 ...
 [ 321   18  416 ...  196    9   12]
 [   0    0    0 ...  706    2   19]
 [   0    0    0 ...    8   11   19]]


In [20]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [21]:
y_train

,sentiment
5478,0
22132,0
33533,1
42605,1
48740,1
...,...
44566,0
30280,0
6637,1
35343,0


In [22]:
y_test

,sentiment
23656,0
27442,0
40162,1
8459,1
8051,1
...,...
44231,0
18034,0
33856,0
15906,1


In [23]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [25]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 268ms/step - accuracy: 0.7174 - loss: 0.5348 - val_accuracy: 0.8418 - val_loss: 0.3814
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 135s 263ms/step - accuracy: 0.8464 - loss: 0.3655 - val_accuracy: 0.7548 - val_loss: 0.4996
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 259ms/step - accuracy: 0.8466 - loss: 0.3636 - val_accuracy: 0.8459 - val_loss: 0.3667
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 131s 262ms/step - accuracy: 0.8676 - loss: 0.3235 - val_accuracy: 0.8621 - val_loss: 0.3559
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 261ms/step - accuracy: 0.8950 - loss: 0.2633 - val_accuracy: 0.8673 - val_loss: 0.3313


In [26]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.8709 - loss: 0.3314
Test Loss: 0.33401525020599365
Test Accuracy: 0.8672999739646912
